## <center><b>ONLINE FOOD DATASET.</b></center>

<b>IMPORT LIBRARY PACKAGES</b>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('always') 
import shap
import pickle

# Model Classifier
from sklearn.svm import SVC
from collections import Counter
from xgboost import XGBClassifier
from sklearn.utils import resample
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from imblearn import under_sampling, over_sampling
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

from flask import Flask, render_template, request

<b>LOAD & DISPLAY DATA</b>

In [ ]:
online_food = pd.read_csv('onlinefoods.csv')
online_food.head()

#### <b> DATA EXPLORATION </b>

CHECK FOR NUMBER OF ROWS AND COLUMNS IN THIS DATASET

In [ ]:
print('The total number of rows and columns is',online_food.shape,'respectively.')

DATASET SUMMARY

This section provides general insight of this dataset.

In [ ]:
online_food.info()

CHECK FOR ANY MISSING VALUES

In [ ]:
online_food.isna()

TOTAL SUM OF MISSING VALUES

In [ ]:
online_food.isna().sum()

CHECK FOR DUPLICATE ROWS

In [ ]:
print ("There are",online_food.duplicated().sum(), "duplicate values in this dataset.")

DISPLAY DUPLICATES ROWS

In [ ]:
duplicates = online_food[online_food.duplicated(keep=False)]
duplicates

INSIGHTS

In this dataset, there were no missing values but 103 entry duplicates were identified.

The duplicates were identified in the following columns: 
- Age 
- Gender 
- Marital Status 
- Occupation 
- Monthly income 
- Educational Qualifications 
- Output and 
- Feedback.

SUMMARY STATISTICS

In [ ]:
online_food.describe(include='all').T

In [ ]:
online_food.boxplot()

#### <b> DATA CLEANING </b>

Under this section, I will be preparing the dataset for my analysis and ensure it meets up to the standard I want it to be.

To get this done, I will do the following:
- Drop unwanted columns
- Rename some columns or entries

DROP UNWANTED COLUMN

In [ ]:
online_food.drop (columns=['Unnamed: 12'], inplace = True)
online_food.head()

#### <b>EXPLORATORY DATA ANALYSIS, (E.D.A)</b>

- A BAR CHART DISPLAYING THE VARIOUS AGES OF THIS DATASET.

In [ ]:
# Extract the values from the 'Age' column and count occurrences
age_counts = online_food['Age'].value_counts().reset_index()
age_counts.columns = ['Age', 'Count']

# Create the bar chart with color grading
fig = px.bar(age_counts, x='Age', y='Count', title='Age Distribution',
             labels={'Age': 'Age', 'Count': 'Number of People'},
             color='Age', color_continuous_scale='Viridis')

# Show the plot
fig.show()

CREATE AN AGE GROUP BASED ON AGE RANGES

In [ ]:
bins = [0, 10, 20, 30, 40]
labels = ['0-10', '11-20', '21-30', '31-40']
online_food['Age Group'] = pd.cut(online_food['Age'], bins=bins, labels=labels)
online_food

MEAN VALUE OF AGE

In [ ]:
mean_value = online_food['Age'].mean()
print('The mean value for age is',mean_value)

MEDIAN VALUE OF AGE

In [ ]:
median_value = online_food['Age'].median()
print('The median value for age is',median_value)

MODE VALUE OF AGE

In [ ]:
mode_value = online_food['Age'].mode().values[0]
print('The median value for age is',mode_value)

CHECK THE TOTAL AND PERCENTAGE OF GENDER

In [ ]:
# Calculate the counts and percentages
gender_counts = online_food['Gender'].value_counts().reset_index()
gender_counts.columns = ['Gender', 'Count']
#gender_counts['Percentage'] *= 100

# Create the donut chart
fig = px.pie(gender_counts, values='Count', names='Gender', 
             title='Gender Distribution', 
             hole=0.3,  # Set the size of the hole for the donut chart
             labels={'Gender': 'Gender', 'Percentage': 'Percentage (%)'},
             color_discrete_sequence=px.colors.qualitative.Pastel)  # Set colors for the segments

# Add data labels
fig.update_traces(textposition='inside', textinfo='percent+label')

# Show the plot
fig.show()

CHECK THE TOTAL AND PERCENTAGE OF MARITAL STATUS

In [ ]:
marital_status_counts = online_food['Marital Status'].value_counts().reset_index()
marital_status_counts.columns = ['Marital Status', 'Count']

#colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA']

# Create the pie chart
fig = px.pie(marital_status_counts, names='Marital Status', 
                values='Count', hole=0.3, 
                title='Marital Status Distribution', 
                color_discrete_sequence=px.colors.qualitative.Pastel)

# Update the chart to add data labels
fig.update_traces(textinfo='percent+label')

# Show the plot
fig.show()

CHECK THE TOTAL AND PERCENTAGE OF OCCUPATION

In [ ]:
# Calculate the counts and percentages
occupation_counts = online_food['Occupation'].value_counts().reset_index()
occupation_counts.columns = ['Occupation', 'Count']
occupation_counts['Percentage'] = ((occupation_counts['Count'] / occupation_counts['Count'].sum()) * 100).round(1)

# Create the bar chart with color grading
fig = px.bar(occupation_counts, x='Occupation', y='Count', title='Occupation Distribution',
             labels={'Occupation': 'Occupation', 'Count': 'Count'},
             color='Occupation', color_continuous_scale='Viridis',
             hover_data={'Percentage': ':.1f%'})

# Add data labels (optional)
fig.update_traces(texttemplate='%{y}', textposition='inside')

# Show the plot
fig.show()

CHECK THE TOTAL AND PERCENTAGE OF MONTHLY INCOME

In [ ]:
# Calculate the counts and percentages
monthly_income_counts = online_food['Monthly Income'].value_counts().reset_index()
monthly_income_counts.columns = ['Monthly Income', 'Count']
monthly_income_counts['Percentage'] = ((monthly_income_counts['Count'] / monthly_income_counts['Count'].sum()) * 100).round(1)

# Create the bar chart with color grading
fig = px.bar(monthly_income_counts, x='Monthly Income', y='Count', title='Monthly Income Distribution',
             labels={'Monthly Income': 'Monthly Income', 'Count': 'Count'},
             color='Monthly Income', color_continuous_scale='Viridis',
             hover_data={'Percentage': ':.1f%'})

# Add data labels (optional)
fig.update_traces(texttemplate='%{y}', textposition='inside')

# Show the plot
fig.show()

CHECK THE TOTAL AND PERCENTAGE OF EDUCATIONAL QUALIFICATIONS

In [ ]:
# Calculate the counts and percentages
edu_qual_counts_counts = online_food['Educational Qualifications'].value_counts().reset_index()
edu_qual_counts.columns = ['Educational Qualifications', 'Count']
edu_qual_counts['Percentage'] = ((edu_qual_counts['Count'] / edu_qual_counts['Count'].sum()) * 100).round(1)

# Create the bar chart with color grading
fig = px.bar(edu_qual_counts, x='Educational Qualifications', y='Count', title='Educational Qualification Distribution',
             labels={'Educational Qualifications': 'Educational Qualifications', 'Count': 'Count'},
             color='Educational Qualifications', color_continuous_scale='Viridis',
             hover_data={'Percentage': ':.1f%'})

# Add data labels (optional)
fig.update_traces(texttemplate='%{y}', textposition='inside')

# Show the plot
fig.show()

CHECK THE TOTAL AND PERCENTAGE OF FAMILY SIZE

In [ ]:
# Calculate the counts and percentages
family_size_counts = online_food['Family size'].value_counts().reset_index()
family_size_counts.columns = ['Family size', 'Count']
family_size_counts['Percentage'] = ((family_size_counts['Count'] / family_size_counts['Count'].sum()) * 100).round(1)

# Create the bar chart with color grading
fig = px.bar(family_size_counts, x='Family size', y='Count', title='Family size Distribution',
             labels={'Family size': 'Family sizes', 'Count': 'Count'},
             color='Family size', color_continuous_scale='Viridis',
             hover_data={'Percentage': ':.1f%'})

# Add data labels (optional)
fig.update_traces(texttemplate='%{y}', textposition='inside')

# Show the plot
fig.show()

In [ ]:
# Extract the values from the 'Age' column and count occurrences
pin_code_counts = online_food['Pin code'].value_counts().reset_index()
pin_code_counts.columns = ['Pin Code', 'Count']

# Create the bar chart with color grading
fig = px.bar(pin_code_counts, x='Pin Code', y='Count', title='Pin Code Distribution',
             labels={'Pin Code': 'Pin Code', 'Count': 'Pin Code'},
             color='Pin Code', color_continuous_scale='Viridis')

# Show the plot
fig.show()

In [ ]:
output_counts = online_food['Output'].value_counts().reset_index()
output_counts.columns = ['Output', 'Count']

#colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA']

# Create the pie chart
fig = px.pie(output_counts, names='Output', 
                values='Count', hole=0.3, 
                title='Output Distribution', 
                color_discrete_sequence=px.colors.qualitative.Pastel)

# Update the chart to add data labels
#fig.update_traces(textinfo='percent+label+value')

# Show the plot
fig.show()

In [ ]:
# Calculate the counts and percentages
feedback_counts = online_food['Feedback'].value_counts().reset_index()
feedback_counts.columns = ['Feedback', 'Percentage']
feedback_counts['Percentage'] *= 100

# Create the donut chart
fig = px.pie(feedback_counts, values='Percentage', names='Feedback', 
             title='Feedback Distribution', 
             hole=0.3,  # Set the size of the hole for the donut chart
             labels={'Feedback': 'Feedback', 'Percentage': 'Percentage (%)'},
             color_discrete_sequence=px.colors.qualitative.Pastel)  # Set colors for the segments

# Add data labels
fig.update_traces(textposition='inside', textinfo='percent+label+value')

# Show the plot
fig.show()

- VISUALIZATION FOR NUMERICAL VARIABLE

In [ ]:
# Visualization (box plot for numerical variable)
plt.figure(figsize=(8,8))
sns.boxplot(online_food['Age'])
plt.title('Box Plot')
plt.xlabel('Age')
plt.show()

- BIVARIATE ANALYSIS

In [ ]:
#Bivariate analysis for 'Age' vs 'Output'
fig = px.box(online_food, x='Age', y='Output', title='Age vs Output')
fig.update_xaxes(title='Age')
fig.update_yaxes(title='Output')
fig.show()


# Bivariate analysis for 'Age' vs 'Marital Status'
fig = px.histogram(online_food, x='Age', color='Marital Status', 
                   title='Age vs Marital Status',
                   labels={'Age': 'Age', 'Marital Status': 'Marital Status', 'count': 'Count'},
                   barmode='group')

fig.update_layout(xaxis_title='Age', yaxis_title='Count')
fig.show()


# Bivariate analysis for 'Age' vs 'Monthly Income'
fig = px.histogram(online_food, x='Age', color='Monthly Income', 
             title='Age vs Monthly Income', 
             labels={'Age': 'Age', 'Monthly Income': 'Monthly Income', 'count': 'Count'})

fig.update_layout(xaxis_title='Age', yaxis_title='Count')
fig.show()


# Bivariate analysis for 'Age' vs 'Occupation'
fig = px.histogram(online_food, x='Age', color='Occupation', 
             title='Age vs Occupation', 
             labels={'Age': 'Age', 'Occupation': 'Occupation', 'count': 'Count'})

fig.update_layout(xaxis_title='Age', yaxis_title='Count')
fig.show()


# Bivariate analysis for 'Gender' vs 'Output'
fig = px.histogram(online_food, x='Gender', color='Educational Qualifications', 
             title='Gender vs Educational Qualifications', 
             labels={'Gender': 'Gender', 'Educational Qualifications': 'Educational Qualifications', 'count': 'Count'})

fig.update_layout(xaxis_title='Gender', yaxis_title='Count')
fig.show()


#Bivariate analysis for 'Marital Status' vs 'Gender'
# Create a cross-tabulation
cross_tab = pd.crosstab(online_food['Gender'], online_food['Marital Status'])

# Plot the heatmap
fig = px.imshow(cross_tab,
                labels=dict(x="Marital Status", y="Gender", color="Count"),
                x=cross_tab.columns,
                y=cross_tab.index,
                color_continuous_scale='viridis')

fig.update_layout(title='Gender vs. Marital Status',
                  xaxis_title='Marital Status',
                  yaxis_title='Gender')

fig.show()


# Bivariate analysis for 'Gender' vs 'Output'
fig = px.histogram(online_food, x='Gender', color='Output', 
                   title='Gender vs Output',
                   labels={'Gender': 'Gender', 'Output': 'Output', 'count': 'Count'},
                   barmode='group')

fig.update_layout(xaxis_title='Gender', yaxis_title='Count')
fig.show()


# Bivariate analysis for 'Marital Status' vs 'Output'
fig = px.histogram(online_food, x='Marital Status', color='Output', 
                   title='Marital Status vs Output',
                   labels={'Marital Status': 'Marital Status', 'Output': 'Output', 'count': 'Count'},
                   barmode='group')

fig.update_layout(xaxis_title='Marital Status', yaxis_title='Count')
fig.show()

# Bivariate analysis for 'Marital Status' vs 'Monthly Income'
fig = px.histogram(online_food, x='Marital Status', color='Monthly Income', 
                   title='Marital Status vs Monthly Income',
                   labels={'Marital Status': 'Marital Status', 'Monthly Income': 'Monthly Income', 'count': 'Count'},
                   barmode='group')

fig.update_layout(xaxis_title='Marital Status', yaxis_title='Monthly Income')
fig.show()


# Bivariate analysis for 'Occupation' vs 'Output'
fig = px.histogram(online_food, x='Occupation', color='Output', 
                   title='Occupation vs Output',
                   labels={'Occupation': 'Occupation', 'Output': 'Output', 'count': 'Count'},
                   barmode='group')

fig.update_layout(xaxis_title='Occupation', yaxis_title='Count')
fig.show()


# Bivariate analysis for 'Educational Qualifications' vs 'Output'
fig = px.histogram(online_food, x='Educational Qualifications', color='Output', 
                   title='Educational Qualifications vs Output',
                   labels={'Educational Qualifications': 'Educational Qualifications', 'Output': 'Output', 'count': 'Count'},
                   barmode='group')

fig.update_layout(xaxis_title='Educational Qualifications', yaxis_title='Count')
fig.show()


# Bivariate analysis for 'Family size' vs 'Output'
fig = px.box(online_food, x='Output', y='Family size', 
             title='Family size vs Output', 
             labels={'Output': 'Output', 'Family size': 'Family size'})

fig.update_xaxes(title='Output')
fig.update_yaxes(title='Family size')
fig.show()


# Bivariate analysis for 'Feedback' vs 'Output'
fig = px.histogram(online_food, x='Feedback', color='Output', 
                   title='Feedback vs Output',
                   labels={'Feedback': 'Feedback', 'Output': 'Output', 'count': 'Count'},
                   barmode='group')

fig.update_layout(xaxis_title='Feedback', yaxis_title='Count')
fig.show()


# Bivariate analysis for 'Monthly Income' vs 'Gender'
fig = px.histogram(online_food, x='Monthly Income', color='Gender', 
                   title='Monthly Income vs Gender',
                   labels={'Monthly Income': 'Monthly Income', 'Gender': 'Gender', 'count': 'Count'},
                   barmode='group')

fig.update_layout(xaxis_title='Educational Qualifications', yaxis_title='Count')
fig.show()

- MULTIVARIATE ANALYSIS

In [ ]:
# Pairplot for numerical variables
# Create the pairplot
g = sns.pairplot(online_food, diag_kind='kde', height=3, aspect=1.5)
g.fig.suptitle('Pairplot of Numerical Variables', y=1.02)

# Adjust plot aesthetics
plt.tight_layout(rect=[0, 0, 1, 0.96])

# Show the plot
plt.show()


# Boxplot of 'Rating' by 'Gender' and 'Marital Status'
plt.figure(figsize=(12, 6))
sns.boxplot(x='Age', y='Marital Status', hue='Family size', data=online_food)
plt.title('Age by Marital Status and Family size')
plt.xlabel('Age')
plt.ylabel('Marital Status')
plt.show()


# Barplot of 'Feedback' by 'Occupation' and 'Educational Qualifications'
plt.figure(figsize=(10, 6))
sns.histplot(x='Occupation', y='Feedback', hue='Educational Qualifications', data=online_food)
plt.title('Feedback by Occupation and Educational Qualifications')
plt.xlabel('Occupation')
plt.ylabel('Feedback')
plt.xticks(rotation=45)
plt.show()


# Barplot of Feedback by Occupation and Educational Qualifications
fig = px.bar(online_food, x='Occupation', y='Feedback', color='Educational Qualifications',
             title='Feedback by Occupation and Educational Qualifications')

fig.update_layout(xaxis_title='Occupation', yaxis_title='Feedback')
fig.update_xaxes(tickangle=45)
fig.show()

#### <b>ANALYSIS QUESTIONS</b>

- What is the distribution of ages among different educational qualification levels

- What is the marital status variation across different occupations

- What is the correlation between educational qualifications and monthly incomes in specific occupations

- What is the variation in feedback sentiment across different age groups

- Which occupation gave the highest value for positive feedback?

- What is the Correlation between family size and feedback?

- Is there any geographic trend in monthly income based on latitude and longitude?

1. What is the distribution of ages among different educational qualification levels

In [ ]:
# Assuming 'online_food' is your DataFrame
fig = px.box(online_food, x='Educational Qualifications', y='Age', 
             title='Distribution of Ages by Educational Qualifications',
             labels={'Educational Qualifications': 'Educational Qualifications', 'Age': 'Age'})

fig.update_layout(xaxis_tickangle=-45)

fig.show()

2. Marital status variation across different occupations

In [ ]:
marital_status_occupation = pd.crosstab(online_food['Occupation'], online_food['Marital Status'])

# Reindex the DataFrame to ensure all marital status categories are present
marital_status_occupation = marital_status_occupation.reindex(columns=['Single', 'Married', 'Divorced'], fill_value=0)

# Reset index to convert the result to a DataFrame
marital_status_occupation.reset_index(inplace=True)

# Melt the DataFrame to long format
melted_online_food = pd.melt(marital_status_occupation, id_vars=['Occupation'], value_vars=['Single', 'Married', 'Divorced'], 
                    var_name='Marital Status', value_name='Count')

# Plot using Plotly Express
fig = px.bar(melted_online_food, x='Occupation', y='Count', color='Marital Status', title='Marital Status Variation Across Occupations',
             labels={'Occupation': 'Occupation', 'Count': 'Count', 'Marital Status': 'Marital Status'},
             barmode='stack')

fig.update_layout(xaxis_tickangle=-45)
fig.show()

3. Correlation between educational qualifications and monthly incomes in specific occupations

In [ ]:
fig = px.scatter(online_food, x='Educational Qualifications', y='Monthly Income', color='Occupation',
                 title='Correlation between Educational Qualifications and Monthly Incomes in Specific Occupations',
                 labels={'Educational Qualifications': 'Educational Qualifications', 'Monthly Income': 'Monthly Income'},
                 category_orders={'Occupation': ['Occupation1', 'Occupation2', 'Occupation3']})

fig.update_layout(xaxis_tickangle=-45)
fig.show()

4. Variation in feedback sentiment across different age groups

In [ ]:
fig = px.histogram(online_food, x='Age Group', color='Feedback', 
             title='Variation in Feedback Sentiment Across Age Groups',
             labels={'Age Group': 'Age Group', 'Feedback': 'Feedback Sentiment', 'count': 'Count'})

fig.show()

5. Which occupation gave the highest value for positive feedback?

In [ ]:
# Filtering the dataset for positive feedback
positive_feedback_data = online_food[online_food['Feedback'] == 'Positive']

# Count the occurrences of each occupation
occupation_counts = positive_feedback_data['Occupation'].value_counts().reset_index()
occupation_counts.columns = ['Occupation', 'Count']

# Create a bar plot using Plotly Express
fig = px.bar(occupation_counts, x='Occupation', y='Count', 
             title='Occupations Among Customers Giving Positive Feedback',
             labels={'Occupation': 'Occupation', 'Count': 'Count'})

# Rotate x-axis labels for better readability
fig.update_layout(xaxis_tickangle=-45)

fig.show()

6. What is the correlation between family size and feedback?

In [ ]:
fig = px.scatter(online_food, x='Family size', y='Feedback', 
                 title='Correlation between Family Size and Feedback Score', 
                 labels={'Family size': 'Family Size', 'Feedback': 'Feedback Score'})

fig.show()

7. Is there any geographic trend in monthly income based on latitude and longitude?

In [ ]:
fig = px.scatter(online_food, x='longitude', y='latitude', 
                 color='Monthly Income', 
                 title='Geographic Trend in Monthly Income', 
                 labels={'longitude': 'Longitude', 'latitude': 'Latitude'},
                 color_continuous_scale='Viridis')

fig.show()

CORRELATION HEATMAP

# Drop the 'Age Group' column
online_food_without_age_group = online_food.drop(columns=['Age Group', 'Gender', 'Marital Status'])

# Heatmap for correlation excluding 'Age Group'
plt.figure(figsize=(10, 10))
correlation_matrix = online_food_without_age_group.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap')
plt.show()

LABEL ENCODING

In [ ]:
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Iterate over categorical columns and apply label encoding
for column in ['Output']:
    online_food[column] = label_encoder.fit_transform(online_food[column].values)

# Display the encoded dataset
online_food

<b> MODELLING </b>

In [ ]:
# Load the dataset
X = online_food[['Age', 'Family size']]
y = online_food['Output']

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define models
models = {
    "Logistic Regression": LogisticRegression(),
    "Random Forest": RandomForestClassifier(),
    "Support Vector Machine": SVC()
}

# Create a pipeline for each model
pipelines = {}
for name, model in models.items():
    pipelines[name] = Pipeline([
        ("scaler", scaler),
        ("model", model)
    ])

# Train and evaluate models using pipelines
for name, pipeline in pipelines.items():
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    classification = classification_report(y_test, y_pred)
    print(f"{name} Metrics:")
    print(f"Accuracy: {accuracy}")
    print(f"Classification Report:\n{classification}\n")

CHECK FOR DATA IMBALANCE

In [ ]:
target_counts = online_food['Output'].value_counts().reset_index()
target_counts.columns = ['Output', 'Count']

# Create the bar plot
fig = px.bar(target_counts, x='Output', y='Count', title='Class Distribution')

# Show the plot
fig.show()

In [ ]:
print('Original dataset shape %s' % Counter(y_train))

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_resampled))

In [ ]:
classifier = RandomForestClassifier(class_weight='balanced')

HYPERPARAMETER TUNING

In [ ]:
# Define the model
model = RandomForestClassifier()

# Define the grid of hyperparameters
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')

# Perform Grid Search
grid_search.fit(X, y)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

#### <b> ONLINE PREDICTION APP </b>

In [ ]:
app = Flask(__prediction.py__)

# Load the pre-trained model
model = pickle.load(open('model.pkl','rb'))

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    # Get user input from the form
    age = int(request.form['age'])
    monthly_income = int(request.form['monthly_income'])
    family_size = int(request.form['family_size'])

    # Make prediction using the model
    prediction = model.predict([[age, monthly_income, family_size]])

    # Map prediction to human-readable output
    output = "Satisfied" if prediction[0] == 1 else "Not Satisfied"

    return render_template('index.html', prediction_text='Customer is {}'.format(output))

if __name__ == '__main__':
    app.run(debug=True)